In [1]:
from datasets import load_dataset

dataset = load_dataset("hungnm/multilingual-amazon-review-sentiment-processed")

/home/nalin/yes/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


desired_features = ['stars', 'text', 'language']
reduced_dataset = dataset.select_columns(desired_features)

In [3]:

test= reduced_dataset['test']


In [4]:
def remap_labels(example):
    class_mapping = {1: 0, 2: 1, 4: 2, 5: 3}
    example['stars'] = class_mapping.get(example['stars'])  
    return example


test = test.map(remap_labels)


In [5]:



de_test=test.filter(lambda example: example['language']=='de')
en_test=test.filter(lambda example: example['language']=='en')
fr_test=test.filter(lambda example: example['language']=='fr')
es_test=test.filter(lambda example: example['language']=='es')
ja_test=test.filter(lambda example: example['language']=='ja')
zh_test=test.filter(lambda example: example['language']=='zh')





In [6]:
columns_needed=['stars', 'text']
de_test = de_test.select_columns(columns_needed)
en_test = en_test.select_columns(columns_needed)
fr_test = fr_test.select_columns(columns_needed)
es_test = es_test.select_columns(columns_needed)
ja_test = ja_test.select_columns(columns_needed)
zh_test = zh_test.select_columns(columns_needed)



In [7]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }



In [8]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
test_dataset = CustomDataset(zh_test['text'], zh_test['stars'], tokenizer, 512)
test_loader = DataLoader(test_dataset, batch_size=16 * 4, shuffle=False)
device = torch.device('cuda')
import torch
from torch.utils.data import DataLoader


# Load the pre-trained model for sequence classification
model = XLMRobertaForSequenceClassification.from_pretrained("/home/nalin/ML_metaQR/trial/XLM/zh_xlm_model").to(device)
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score

test_predicted = []
test_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing", unit="batch"):
        inputs = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=inputs, attention_mask=attention_mask)
        logits = outputs.logits

        _, predicted = torch.max(logits, 1)
        test_predicted.extend(predicted.tolist())
        test_labels.extend(labels.tolist())

test_f1 = f1_score(test_labels, test_predicted, average='weighted')
accuracy = accuracy_score(test_labels, test_predicted)

# Print test F1 score and accuracy
print(f"Test F1 Score: {test_f1}")
print(f"Test Accuracy: {accuracy}")

Testing: 100%|██████████| 125/125 [01:01<00:00,  2.02batch/s]

Test F1 Score: 0.6135931706514932
Test Accuracy: 0.61625


In [9]:

test_dataset = CustomDataset(fr_test['text'], fr_test['stars'], tokenizer, 512)
test_loader = DataLoader(test_dataset, batch_size=16 * 4, shuffle=False)
device = torch.device('cuda')
import torch
from torch.utils.data import DataLoader


# Load the pre-trained model for sequence classification
model = XLMRobertaForSequenceClassification.from_pretrained("/home/nalin/ML_metaQR/trial/XLM/fr_xlm_model").to(device)
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score

test_predicted = []
test_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing", unit="batch"):
        inputs = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=inputs, attention_mask=attention_mask)
        logits = outputs.logits

        _, predicted = torch.max(logits, 1)
        test_predicted.extend(predicted.tolist())
        test_labels.extend(labels.tolist())

test_f1 = f1_score(test_labels, test_predicted, average='weighted')
accuracy = accuracy_score(test_labels, test_predicted)

# Print test F1 score and accuracy
print(f"Test F1 Score: {test_f1}")
print(f"Test Accuracy: {accuracy}")

Testing: 100%|██████████| 124/124 [01:02<00:00,  1.99batch/s]

Test F1 Score: 0.6120967752250155
Test Accuracy: 0.6204241353193638


In [10]:

test_dataset = CustomDataset(de_test['text'], de_test['stars'], tokenizer, 512)
test_loader = DataLoader(test_dataset, batch_size=16 * 4, shuffle=False)
device = torch.device('cuda')
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

# Load the pre-trained model for sequence classification
model = XLMRobertaForSequenceClassification.from_pretrained("/home/nalin/ML_metaQR/trial/XLM/de_xlm_model").to(device)
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score

test_predicted = []
test_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing", unit="batch"):
        inputs = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=inputs, attention_mask=attention_mask)
        logits = outputs.logits

        _, predicted = torch.max(logits, 1)
        test_predicted.extend(predicted.tolist())
        test_labels.extend(labels.tolist())

test_f1 = f1_score(test_labels, test_predicted, average='weighted')
accuracy = accuracy_score(test_labels, test_predicted)

# Print test F1 score and accuracy
print(f"Test F1 Score: {test_f1}")
print(f"Test Accuracy: {accuracy}")

Testing: 100%|██████████| 123/123 [01:02<00:00,  1.98batch/s]

Test F1 Score: 0.6494285124229336
Test Accuracy: 0.6518971224853578


In [11]:

test_dataset = CustomDataset(ja_test['text'], ja_test['stars'], tokenizer, 512)
test_loader = DataLoader(test_dataset, batch_size=16 * 4, shuffle=False)
device = torch.device('cuda')
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

# Load the pre-trained model for sequence classification
model = XLMRobertaForSequenceClassification.from_pretrained("/home/nalin/ML_metaQR/trial/XLM/ja_xlm_model").to(device)
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score

test_predicted = []
test_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing", unit="batch"):
        inputs = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=inputs, attention_mask=attention_mask)
        logits = outputs.logits

        _, predicted = torch.max(logits, 1)
        test_predicted.extend(predicted.tolist())
        test_labels.extend(labels.tolist())

test_f1 = f1_score(test_labels, test_predicted, average='weighted')
accuracy = accuracy_score(test_labels, test_predicted)

# Print test F1 score and accuracy
print(f"Test F1 Score: {test_f1}")
print(f"Test Accuracy: {accuracy}")

Testing: 100%|██████████| 125/125 [01:03<00:00,  1.97batch/s]

Test F1 Score: 0.6037795836730133
Test Accuracy: 0.6169521190148769


In [12]:

test_dataset = CustomDataset(es_test['text'], es_test['stars'], tokenizer, 512)
test_loader = DataLoader(test_dataset, batch_size=16 * 4, shuffle=False)
device = torch.device('cuda')
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

# Load the pre-trained model for sequence classification
model = XLMRobertaForSequenceClassification.from_pretrained("/home/nalin/ML_metaQR/trial/XLM/es_xlm_model").to(device)
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score

test_predicted = []
test_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing", unit="batch"):
        inputs = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=inputs, attention_mask=attention_mask)
        logits = outputs.logits

        _, predicted = torch.max(logits, 1)
        test_predicted.extend(predicted.tolist())
        test_labels.extend(labels.tolist())

test_f1 = f1_score(test_labels, test_predicted, average='weighted')
accuracy = accuracy_score(test_labels, test_predicted)

# Print test F1 score and accuracy
print(f"Test F1 Score: {test_f1}")
print(f"Test Accuracy: {accuracy}")

Testing: 100%|██████████| 124/124 [01:01<00:00,  2.01batch/s]

Test F1 Score: 0.6069780752584067
Test Accuracy: 0.6155403695267021
